In [10]:
from pystac_client import Client
from odc.stac import load

import folium
import plotly.express as px

import geopandas as gpd
import pandas as pd
import numpy as np
import os

from tqdm import tqdm

In [11]:
def search_satellite_images(collection:str, bbox:list[int], date:str, cloud_cover:tuple[int]):
    """
    Utilizes the client https://earth-search.aws.element84.com/v1 to search for satellite images based on
    collection, bounding box, date range, and cloud cover.

    ----------
    Parameters
    ----------
    collection: Collection name, such as sentinel-2-l2a
    bbox: Bounding box [min_lon, min_lat, max_lon, max_lat]
    date: Date range "YYYY-MM-DD/YYYY-MM-DD"
    cloud_cover: Tuple representing cloud cover range (min, max)
    
    ------
    Output
    ------
    Data loaded based on search criteria.
    """
    
    client=Client.open("https://earth-search.aws.element84.com/v1")
    search = client.search(collections=[collection],
                            bbox=bbox,
                            datetime=date,
                            query=[f"eo:cloud_cover<{cloud_cover[1]}", f"eo:cloud_cover>{cloud_cover[0]}"])

    data = load(search.items(), bbox=bbox, groupby="solar_day", chunks={})

    return data

In [12]:
def count_water_pixels(data,lake_id:str) -> pd.DataFrame:
    """
    Counts water pixels from Sentinel-2 SCL data for each time step.

    ----------
    Parameters
    ----------
    data: xarray Dataset with Sentinel-2 SCL data
    id: integer lake id

    ------
    Output
    ------
    DataFrame with dates, water counts, and snow counts.
    """

    water_counts = []
    date_labels = []
    water_area =[]
    coverage_ratio=[]

    # Determine the number of time steps
    numb_days = len(data.time)

    # Iterate through each time step
    for t in range(numb_days):
        scl_image = data[["scl"]].isel(time=t).to_array()
        dt = pd.to_datetime(scl_image.time.values)
        year = dt.year
        month = dt.month
        day = dt.day

        date_string = f"{year}-{month:02d}-{day:02d}"

        # Count the number of pixels corresponding to water
        count_water = np.count_nonzero(scl_image == 6)  # Water

        surface_area = count_water*10*10/(10**6)

        count_pixels = np.count_nonzero((scl_image == 1) | (scl_image == 2) | (scl_image == 3)|(scl_image == 4) | (scl_image == 5) |
                                        (scl_image == 6) | (scl_image == 7) |(scl_image == 8) |(scl_image == 9) |(scl_image == 10) |(scl_image == 11))
        total_pixels = data.sizes['y']*data.sizes['x']

        coverage = count_pixels*10*10/1e6
        lake_area = total_pixels*10*10/1e6

        ratio=coverage/lake_area

        if ratio <0.9:
          continue


        # Append
        water_counts.append(count_water)
        date_labels.append(date_string)
        water_area.append(surface_area)
        coverage_ratio.append(ratio)


    # Convert date labels to pandas datetime format
    datetime_index = pd.to_datetime(date_labels)

    # Create a dictionary for constructing the DataFrame
    data_dict = {
        'Date': datetime_index,
        'ID': lake_id,
        'Water Counts': water_counts,
        'Pixel Counts': count_pixels,
        'Total Pixels': total_pixels,
        'Coverage Ratio': coverage_ratio,
        'Water Surface Area': water_area
    }

    # Create the DataFrame
    df = pd.DataFrame(data_dict)

    return df

In [13]:
def get_centroids_and_bboxes(shapefile_paths:list[str]) -> pd.DataFrame:
    """
    Processes a list of shapefiles to return a DataFrame containing the ID, centroid,
    and bounding box (bbox) of each polygon.

    ----------
    Parameters
    ----------
    shapefile_path: Path to the shapefile.

    ------
    Output
    ------
    DataFrame with the ID, centroid, and bbox of each polygon.
    """
    df = pd.DataFrame()
    for shapefile_path in shapefile_paths:
        # Load the shapefile
        gdf = gpd.read_file(shapefile_path)

        # Reproject to EPSG:4326
        gdf_proj = gdf.to_crs("EPSG:4326")

        
        centroids = []
        bboxes = []

        # Process each polygon to get its centroid and bbox
        for index, row in gdf_proj.iterrows():
            # Centroid
            centroid_lat = row.geometry.centroid.y
            centroid_lon = row.geometry.centroid.x
            centroids.append((centroid_lat, centroid_lon))

            # Bounding Box
            minx, miny, maxx, maxy = row.geometry.bounds
            bbox = (minx, miny, maxx, maxy)
            bboxes.append(bbox)

        # Create the DataFrame
        df_temp = pd.DataFrame({
                    'ID': os.path.split(shapefile_path)[1][:-4].replace('_',' '),
                    'Centroid_Lat': [lat for lat, lon in centroids],
                    'Centroid_Lon': [lon for lat, lon in centroids],
                    'BBox_Min_Lon': [bbox[0] for bbox in bboxes],
                    'BBox_Min_Lat': [bbox[1] for bbox in bboxes],
                    'BBox_Max_Lon': [bbox[2] for bbox in bboxes],
                    'BBox_Max_Lat': [bbox[3] for bbox in bboxes]
                                })
        
        df = pd.concat([df,df_temp])

    return df.reset_index(drop=True)



In [20]:
def plot_timeseries_for_polygon(spot_id:str, ts_df:pd.DataFrame) -> str:
    """
    Function to plot time series for lake location and save as HTML

    ----------
    Parameters
    ----------
    spot_id: Path to the shapefile.
    ts_df: 

    ------
    Output
    ------
    File path string
    """
    df_spot = ts_df[ts_df['ID'] == spot_id]

    fig = px.line(df_spot, x='Date', y='Water Surface Area', title=f'Time Series for {spot_id}')

     # Add X and Y axis labels
    fig.update_layout(
        xaxis_title="Date",
        yaxis_title="Water Surface Area (sq km)"
    )

    filepath = f'tmp_' + spot_id + '.html'
    fig.write_html(filepath, include_plotlyjs='cdn')
    return filepath

In [15]:
shape_folder = r'Shape_Files'
shapefiles = [os.path.join(shape_folder,file) for file in os.listdir(shape_folder) if file.endswith('.shp')]
lakes_df = get_centroids_and_bboxes(shapefiles)
lakes_df

,ID,Centroid_Lat,Centroid_Lon,BBox_Min_Lon,BBox_Min_Lat,BBox_Max_Lon,BBox_Max_Lat
0,Lake Aviemore,-44.624457,170.307621,170.246741,-44.664206,170.365163,-44.594507
1,Lake Benmore,-44.398250,170.222919,170.185929,-44.450288,170.269145,-44.339142
2,Lake Coleridge,-43.294016,171.501483,171.436722,-43.351916,171.589710,-43.231242
3,Lake Hakapoua,-46.170599,166.941473,166.913600,-46.201827,166.968011,-46.134887
4,Lake Hauroko,-45.956769,167.278655,167.136042,-46.079634,167.419615,-45.827298
5,Lake Hawea,-44.471397,169.316510,169.192464,-44.612281,169.458754,-44.288760
6,Lake Manapouri,-45.519208,167.484470,167.270147,-45.594197,167.636295,-45.439678
7,Lake Monowai,-45.860701,167.432154,167.340240,-45.909533,167.535476,-45.808895
8,Lake Ohau,-44.237591,169.862531,169.809539,-44.297465,169.944604,-44.162615
9,Lake Poteriteri,-46.069770,167.114056,167.085472,-46.190528,167.155245,-45.925676


In [22]:
all_water_pixels_dfs = [] 

for lake_id in tqdm(lakes_df.ID):
    
    lake_df = lakes_df[lakes_df['ID'] == lake_id]

    if not lake_df.empty:
        bbox = [lake_df.iloc[0].BBox_Min_Lon, lake_df.iloc[0].BBox_Min_Lat,
                lake_df.iloc[0].BBox_Max_Lon, lake_df.iloc[0].BBox_Max_Lat]

        data = search_satellite_images(collection="sentinel-2-l2a",
                                       date="2023-01-01/2023-12-30",
                                       cloud_cover=(0, 5),
                                       bbox=bbox)
        # Pass the lake_id
        water_pixels_df = count_water_pixels(data, lake_id)

        # Append
        all_water_pixels_dfs.append(water_pixels_df)

# Concatenate all DataFrames into a single DataFrame
df = pd.concat(all_water_pixels_dfs, ignore_index=True)
df.head()

100%|██████████| 20/20 [02:57<00:00,  8.86s/it]


,Date,ID,Water Counts,Pixel Counts,Total Pixels,Coverage Ratio,Water Surface Area
0,2023-01-08,Lake Aviemore,244824,741501,741501,1.0,24.4824
1,2023-01-18,Lake Aviemore,244920,741501,741501,1.0,24.4920
2,2023-02-12,Lake Aviemore,244700,741501,741501,1.0,24.4700
3,2023-04-26,Lake Aviemore,245592,741501,741501,1.0,24.5592
4,2023-06-02,Lake Aviemore,244708,741501,741501,1.0,24.4708


In [23]:
# Create a map
m = folium.Map(location=[-44.70,168.98], zoom_start=7)

# Add markers with Plotly time series popups
for index, row in lakes_df.iterrows():
    html_path = plot_timeseries_for_polygon(row['ID'], df)
    iframe = folium.IFrame(html=open(html_path).read(), width=500, height=300)
    popup = folium.Popup(iframe, max_width=2650)
    folium.Marker([row['Centroid_Lat'], row['Centroid_Lon']], popup=popup).add_to(m)

m.save('NZ_map_of_Lake_Surface_Area.html')

In [24]:
# Clean up temporary HTML files
for spot_id in lakes_df['ID']:
    os.remove(f'tmp_' + spot_id + '.html')